In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

We first enter the input data

In [3]:
fn = '/kaggle/input/open-problems-single-cell-perturbations/de_train.parquet'
df_de_train = pd.read_parquet(fn)# , index_col = 0)
print(df_de_train.shape)
df_de_train

(614, 18216)


,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,0.786192,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,-0.069476,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


Check the target value

In [4]:
gene_expression = df_de_train.iloc[:,5:]
gene_expression.head(5)

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


PCA the differen tial expression data

In [5]:
from sklearn.decomposition import PCA

expression_pca = PCA(n_components=150)
expression_pca.fit_transform(gene_expression)
reduced = expression_pca.transform(gene_expression)

In [8]:
df_transformed = pd.DataFrame(data=reduced, columns=[f"ExpressionPC{i+1}" for i in range(reduced.shape[1])])
pca_result = pd.concat([df_de_train.iloc[:,:5], df_transformed], axis=1)
pca_result

,cell_type,sm_name,sm_lincs_id,SMILES,control,ExpressionPC1,ExpressionPC2,ExpressionPC3,ExpressionPC4,ExpressionPC5,...,ExpressionPC141,ExpressionPC142,ExpressionPC143,ExpressionPC144,ExpressionPC145,ExpressionPC146,ExpressionPC147,ExpressionPC148,ExpressionPC149,ExpressionPC150
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-36.453699,-11.978529,-10.325804,-3.105771,-7.038439,...,-0.311857,-0.032420,-2.979242,2.838817,1.585992,1.946107,2.348853,-2.403457,0.551709,-1.717113
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-56.183083,-7.023339,-10.385958,-3.107539,-4.220357,...,-1.500425,0.243159,-0.813642,0.318168,-4.709615,3.956105,-0.547201,-0.171939,4.332059,-0.567801
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-101.187397,-9.699070,-13.033234,0.981408,-9.048147,...,-0.297034,-0.047154,1.183053,-0.024337,-0.731966,-1.763017,-1.142200,0.046121,2.480088,-1.024124
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-8.525405,-16.438621,-6.131585,-10.634362,14.536690,...,-0.784717,0.524233,-1.082930,-1.084241,1.634890,-0.737326,1.194599,-0.990615,0.911027,0.068218
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,77.322471,-8.243784,-6.747717,-1.243601,-42.172469,...,-4.944013,2.813095,-9.938161,-14.732392,-3.753173,-0.911806,2.520181,14.881505,3.365167,-2.129214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-118.674501,-10.008553,-17.696765,5.826348,-25.944437,...,-0.796127,-0.073731,3.466104,-2.234694,-4.605263,6.566553,-0.973684,-2.712421,-3.696354,-4.617851
610,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-50.344590,-15.266530,-7.485014,-4.040036,-3.706213,...,0.454406,-2.657863,0.844421,1.721222,-0.878258,-2.818185,-0.032295,-1.158130,-1.095205,3.415747
611,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-70.444165,-9.466616,-8.302391,-4.921166,-2.106416,...,1.904016,1.378827,-0.544094,4.187828,-2.105765,-1.158132,0.156445,0.147607,-1.837244,1.590474
612,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-118.315277,-9.388081,-11.780419,3.913271,-13.477420,...,-2.297843,1.282231,-0.054831,0.673179,-3.213213,2.269499,1.143348,1.449313,-0.190055,1.813716


In [19]:
file = '/kaggle/input/open-problems-single-cell-perturbations/id_map.csv'
df_de_predict = pd.read_csv(file)[["cell_type","sm_name"]]
df_de_predict.head()

,cell_type,sm_name
0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,B cells,ABT-199 (GDC-0199)
2,B cells,ABT737
3,B cells,AMD-070 (hydrochloride)
4,B cells,AT 7867


In [15]:
! pip install catboost

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/09/cb/4e1e986adfe82000a3d4f2b3d4e03e7f028b6c486946f34d0ca5ea94dffd/catboost-1.2.2-cp39-cp39-manylinux2014_x86_64.whl.metadata
  Using cached catboost-1.2.2-cp39-cp39-manylinux2014_x86_64.whl.metadata (1.2 kB)
Using cached catboost-1.2.2-cp39-cp39-manylinux2014_x86_64.whl (98.7 MB)


In [28]:
import catboost
from catboost import CatBoostRegressor, Pool
categorical_features = ['cell_type','sm_name']
model = CatBoostRegressor(iterations=100,  # Number of boosting iterations
                          depth=6,        # Depth of the trees
                          learning_rate=0.1,  # Learning rate
                          loss_function='RMSE',  # Specify your loss function (e.g., RMSE for regression)
                          cat_features=categorical_features,  # Categorical features
                          verbose=0)  # Set verbose to 0 to suppress output

In [31]:
Y_reduced_submit = np.zeros( (len(df_de_predict) , reduced.shape[1] )   )
df_train = df_de_train[['cell_type','sm_name']]
for k in range(reduced.shape[1]):
    train_data = Pool(data=df_train, 
                      label=reduced[:,k],
                      cat_features=categorical_features)
    
    test_data = Pool(data=df_de_predict[["cell_type","sm_name"]], 
                      cat_features=categorical_features)
    
    model.fit(train_data)    
    
    predicted_value = model.predict(test_data)
    
    Y_reduced_submit[:,k] = predicted_value

In [32]:
Y_submit = expression_pca.inverse_transform( Y_reduced_submit)
print(Y_submit)

[[ 2.33324073  1.3203142   1.9910517  ... -0.02094141  0.65483198
   0.19572723]
 [ 0.97404827  0.44616289  0.48678306 ... -0.57382027 -0.00922792
  -0.0743949 ]
 [ 2.07943363  0.48137563 -0.58920214 ... -0.33900936 -0.44434091
   0.17343968]
 ...
 [ 0.36726422  0.29390223 -0.18205072 ... -0.65769205 -0.35095649
  -0.30778876]
 [ 2.33324073  1.3203142   1.9910517  ... -0.02094141  0.65483198
   0.19572723]
 [ 0.36726422  0.29390223 -0.18205072 ... -0.65769205 -0.35095649
  -0.30778876]]


In [33]:
df_submit = pd.DataFrame(Y_submit, columns = df_de_train.columns[5:])
df_submit.index.name = 'id'
print( df_submit.shape )
display(df_submit)
df_submit.to_csv('cat_submission.csv')

(255, 18211)


,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,2.333241,1.320314,1.991052,5.052714,8.719804,2.245859,0.650815,1.508585,0.211547,-0.443932,...,-0.018185,1.558842,0.011632,1.712672,3.571878,2.208338,0.988953,-0.020941,0.654832,0.195727
1,0.974048,0.446163,0.486783,1.433878,6.136818,2.974582,0.158991,0.852047,0.086312,-0.570677,...,-0.427661,0.367514,-0.032381,0.488641,1.915636,0.735383,0.402691,-0.573820,-0.009228,-0.074395
2,2.079434,0.481376,-0.589202,3.402114,6.376463,5.128014,0.192628,0.925262,-0.716058,-0.137225,...,-1.181136,0.228427,0.707789,0.636568,2.871060,1.695364,0.763318,-0.339009,-0.444341,0.173440
3,0.974048,0.446163,0.486783,1.433878,6.136818,2.974582,0.158991,0.852047,0.086312,-0.570677,...,-0.427661,0.367514,-0.032381,0.488641,1.915636,0.735383,0.402691,-0.573820,-0.009228,-0.074395
4,2.333241,1.320314,1.991052,5.052714,8.719804,2.245859,0.650815,1.508585,0.211547,-0.443932,...,-0.018185,1.558842,0.011632,1.712672,3.571878,2.208338,0.988953,-0.020941,0.654832,0.195727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,-0.172867,-0.032850,-0.186952,0.522495,3.668897,0.526250,0.087772,0.228039,-0.103806,-0.381443,...,-0.885499,0.076274,-0.038000,-0.334352,1.356414,-0.207554,0.101323,-0.725733,-0.363280,-0.072374
251,2.333241,1.320314,1.991052,5.052714,8.719804,2.245859,0.650815,1.508585,0.211547,-0.443932,...,-0.018185,1.558842,0.011632,1.712672,3.571878,2.208338,0.988953,-0.020941,0.654832,0.195727
252,0.367264,0.293902,-0.182051,0.361462,4.276538,0.763902,0.231650,0.569025,-0.028949,-0.116321,...,-0.209169,-0.080739,0.269882,-0.226978,1.384238,0.270371,0.079506,-0.657692,-0.350956,-0.307789
